In [1]:
#обработка музыки
import librosa

# загрузка mp3 файлов
import audioread.ffdec  # Use ffmpeg decoder
import ffmpeg

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = (20,10)

#файлы в директории
from glob import glob

#воспроизведение музыки
from IPython.display import Audio

In [2]:
# путь до директории с музыкой
music_dir = './music'

#имена всех музыкальных файлов
musicfiles = glob(music_dir+ "\*.mp3")

In [3]:
#загрузка файла
aro = audioread.ffdec.FFmpegAudioFile(musicfiles[0])
y, sr = librosa.load(aro)

In [4]:
#весёлая нарезка детей
#количество элементов для 7.5 секунд
k = librosa.time_to_samples(7.5, sr=sr)
music_fragments=[]
#нарезка
for i in range(int(np.ceil(y.shape[0] / k))):
    music_fragments.append(y[i * k: (i+1) * k])
#приведение последнего объекта к 7.5 секундам
music_fragments[-1] = np.append(music_fragments[-1], np.zeros((k- music_fragments[-1].shape)))

In [5]:
#вызов музыки
Audio(data= music_fragments[-1], rate=sr)

In [6]:
#добавление шума
STD_n= 0.01
noise=np.random.normal(0, STD_n, k)

music_noise = music_fragments[0]+noise

In [7]:
#вызов музыки c шумом
Audio(data= music_noise,  rate=sr)

In [ ]:
import soundfile as sf

In [13]:
save_dir = './data/'
# сохранение музыки
sf.write(save_dir+'example.wav', music_noise, sr)